# Intro ML - Project 2

## Load data

In [8]:
import numpy as np
import pandas as pd
train_data = pd.read_csv('train_features.csv')
labels = pd.read_csv('train_labels.csv')
test_data = pd.read_csv('test_features.csv')

## Data preprocessing

In [2]:
def calculate_time_features(data, n_samples):
    x = []
    features = [np.nanmedian, np.nanmean, np.nanvar, np.nanmin,
           np.nanmax]
    for index in range(int(data.shape[0] / n_samples)):
        patient_data = data[n_samples * index:n_samples * (index + 1), 2:]
        feature_values = np.empty((len(features), train_data_numpy[:, 2:].shape[1]))
        for i, feature in enumerate(features):
            feature_values[i] = feature(patient_data, axis=0)
        x.append(feature_values.ravel())
    ids = data[0::n_samples, 0].astype(np.uint16).copy()
    bits = np.unpackbits(
        np.expand_dims(ids, axis=1).view(np.uint8),
            axis=1)
    x = np.hstack((bits, np.array(x)))
    return x

In [9]:
x_train = calculate_time_features(train_data.to_numpy(), 12)
x_test = calculate_time_features(test_data.to_numpy(), 12)

/home/yardenas/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)
/home/yardenas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
/home/yardenas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Degrees of freedom <= 0 for slice.
  if __name__ == '__main__':
/home/yardenas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: All-NaN slice encountered
  if __name__ == '__main__':


## Learning Pipeline - Subtask 1

In [4]:
labels_ids = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST',
         'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
y_train = labels[labels_ids].to_numpy()

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import (GridSearchCV,
    cross_val_score, KFold)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# TODO (yarden): compare w-w/o scaler,
# diffferent imputing strategies and without patient id feature.
pipeline = make_pipeline(
                    SimpleImputer(strategy='median'),
#                     StandardScaler(),
                    OneVsRestClassifier(
                        GradientBoostingClassifier(subsample=0.5)))
parameter_space = {
    'onevsrestclassifier__estimator__subsample': [0.25, 0.5, 1.0]
}
inner_cv = KFold(n_splits=5, shuffle=True)
outer_cv = KFold(n_splits=5, shuffle=True)
classifier = GridSearchCV(pipeline, parameter_space,
                               n_jobs=-1, scoring='roc_auc',
                               iid=True,
                               refit=True,
                               cv=inner_cv,
                               verbose=True)
scores = cross_val_score(classifier, x_train, y_train,
                            cv=outer_cv,
                            scoring='roc_auc')
print("Cross-validation score is {score:.3f},"
      " standard deviation is {err:.3f}"
      .format(score = scores.mean(), err = scores.std()))
classifier = classifier.fit(x_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 14.5min finished


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 14.7min finished


Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 14.9min finished


Cross-validation score is 0.813, standard deviation is 0.005
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 19.9min finished


ValueError: X has 10 features per sample, expected 191

In [22]:
print(test_data[test_data['pid'] == 1])
print(train_data[train_data['pid'] == 1])
print(test_data.shape)
print(train_data.shape)

Empty DataFrame
Columns: [pid, Time, Age, EtCO2, PTT, BUN, Lactate, Temp, Hgb, HCO3, BaseExcess, RRate, Fibrinogen, Phosphate, WBC, Creatinine, PaCO2, AST, FiO2, Platelets, SaO2, Glucose, ABPm, Magnesium, Potassium, ABPd, Calcium, Alkalinephos, SpO2, Bilirubin_direct, Chloride, Hct, Heartrate, Bilirubin_total, TroponinI, ABPs, pH]
Index: []

[0 rows x 37 columns]
    pid  Time   Age  EtCO2  PTT   BUN  Lactate  Temp  Hgb  HCO3  ...  \
0     1     3  34.0    NaN  NaN  12.0      NaN  36.0  8.7  24.0  ...   
1     1     4  34.0    NaN  NaN   NaN      NaN  36.0  NaN   NaN  ...   
2     1     5  34.0    NaN  NaN   NaN      NaN  36.0  NaN   NaN  ...   
3     1     6  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
4     1     7  34.0    NaN  NaN   NaN      NaN   NaN  NaN   NaN  ...   
5     1     8  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
6     1     9  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
7     1    10  34.0    NaN  NaN   NaN      NaN  37.0  NaN 